In [169]:
%matplotlib inline
import os
import pandas as pd

from bokeh.io import output_notebook, show
from bokeh.layouts import column
from bokeh.plotting import figure

from IPython import display

In [170]:
output_notebook()

Loading BokehJS ...

In [171]:
def fpath_output():
    return os.path.join('/chia_cluster/home/jackyu/radiation/crd',
                        'LW/examples/separate_g_groups',
                        'study__lblnew_g1_threshold',
                        'test_transf_subroutine',
                        'output_testtransf.csv')



def load_output_data():
    fpath = fpath_output()
    df = pd.read_csv(fpath, sep=r'\s+')
    return df

In [172]:
def plt_tku(df):
    p = figure(title='tku', plot_width=600, plot_height=300)
    p.circle(df['in_logdx'], df['out_tku'], color='red')
    p.line(df['in_logdx'], df['out_tku'])
    p.yaxis.axis_label = 'out_tku'
    p.xaxis.axis_label = 'in_logdx'
    return p



def plt_slope(df):
    p = figure(title='slope', plot_width=600, plot_height=300)
    p.square(df['in_logdx'], df['out_tku_slope'], color='red')
    p.line(df['in_logdx'], df['out_tku_slope'])
    p.yaxis.axis_label = 'out_tku_slope' 
    p.xaxis.axis_label = 'in_logdx'
    return p



def make_fig(df):
    p_tku = plt_tku(df)
    p_slope = plt_slope(df)
    p = column(p_tku, p_slope)
    return p

In [173]:
df = load_output_data()

In [174]:
print('Basic stats')
df.describe()

Basic stats


,i,in_dx,in_logdx,out_tku,out_tku_slope,alert
count,1601.000000,1.601000e+03,1601.000000,1601.000000,1.601000e+03,1601.0
mean,801.000000,5.456581e-01,-3.000000,0.815966,-6.246095e-04,0.0
std,462.313206,1.564539e+00,2.311566,0.331400,1.135705e-03,0.0
min,1.000000,1.000000e-07,-7.000000,0.000000,-8.417005e-03,0.0
25%,401.000000,1.000000e-05,-5.000000,0.832625,-6.240030e-04,0.0
50%,801.000000,1.000000e-03,-3.000000,0.998014,-1.192718e-05,0.0
75%,1201.000000,1.000000e-01,-1.000000,0.999980,-1.195449e-07,0.0
max,1601.000000,1.000000e+01,1.000000,1.000000,0.000000e+00,0.0


In [175]:
print('Where there is +ve slope')
df[df['out_tku_slope'] > 0]

Where there is +ve slope


,i,in_dx,in_logdx,out_tku,out_tku_slope,alert


In [176]:
p = make_fig(df)

In [177]:
show(p)
print('Figure. Output and slope.')

Figure. Output and slope.


In [178]:
df.to_csv('output_test_transf.csv')

In [179]:
!pwd

/chia_cluster/home/jackyu/radiation/offline_radiation_notebooks/longwave/lblnew_20160916/study__g1_threshold


In [180]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')
